In [1]:
import pandas as pd
import matplotlib.pylab as plt
import statsmodels.api as sm
import numpy as np
import seaborn as sns
from scipy import stats

### 사기 혐의자 데이터 읽기

In [3]:
read_path = '../csv/tmp/fraud_uid.csv'  # 8.fraud_find_variables 에서 작업 함
fraud_uid = pd.read_csv(read_path)

fraud_uid.tail()

,uid,help,block,block_by_operation,black_source
212508,6534153,0.0,0.0,1.0,operation
212509,2610325,0.0,0.0,1.0,operation
212510,3927135,0.0,0.0,1.0,operation
212511,1806746,0.0,0.0,1.0,operation
212512,8970553,0.0,0.0,1.0,operation


## black user와 white user의 이름 변경 횟수 T-Test

In [4]:
read_path = '../csv/changed_name_raw.csv'
fraud_name_raw = pd.read_csv(read_path)

fraud_name_raw.tail()

,date,uid
5823231,2019-07-04 10:53:31,9314176
5823232,2019-07-04 10:53:40,6263122
5823233,2019-07-04 10:53:44,6590985
5823234,2019-07-04 10:54:22,9325264
5823235,2019-07-04 10:54:29,3305677


In [5]:
fraud_name = fraud_name_raw.groupby(['uid'], as_index=False).agg({'date': 'count'})

fraud_name = fraud_name.rename(columns={'date': 'changed'})
fraud_name['uid'] = fraud_name['uid'].astype(str)

fraud_name.tail()

,uid,changed
4099311,9325552,1
4099312,9325572,1
4099313,9325578,1
4099314,9325579,1
4099315,9325602,1


In [6]:
fraud_name_uid = pd.merge(fraud_name, fraud_uid, on=['uid'], how='left')

fraud_name_uid = fraud_name_uid.drop(['black_source'], axis=1).fillna(0)

fraud_name_uid.tail()

,uid,changed,help,block,block_by_operation
4102690,9325552,1,0.0,0.0,0.0
4102691,9325572,1,0.0,0.0,0.0
4102692,9325578,1,0.0,0.0,0.0
4102693,9325579,1,0.0,0.0,0.0
4102694,9325602,1,0.0,0.0,0.0


### help, 상담센터 신고된 유저

In [7]:
fraud_name_uid_normal = fraud_name_uid[fraud_name_uid['help'] == 0]
fraud_name_uid_black = fraud_name_uid[fraud_name_uid['help'] == 1]

In [13]:
# 샘플 수 비교
np.size(fraud_name_uid_normal), np.size(fraud_name_uid_black)

(20022460, 491015)

In [14]:
# 평균 비교
np.mean(fraud_name_uid_normal['changed']), np.mean(fraud_name_uid_black['changed'])

(1.3953342895927874, 2.4676741036424548)

In [15]:
# 등분산 검정
leveneTest = stats.levene(fraud_name_uid_normal['changed'], fraud_name_uid_black['changed'])

leveneTest

LeveneResult(statistic=66411.07416078707, pvalue=0.0)

In [16]:
# T-test
tTestResult = stats.ttest_ind(fraud_name_uid_normal['changed'], fraud_name_uid_black['changed'], equal_var=False)

tTestResult

Ttest_indResult(statistic=-124.22704575665718, pvalue=0.0)

### block, 영구 제재된 유저

In [17]:
fraud_name_uid_normal = fraud_name_uid[fraud_name_uid['block'] == 0]
fraud_name_uid_black = fraud_name_uid[fraud_name_uid['block'] == 1]

In [18]:
# 샘플 수 비교
np.size(fraud_name_uid_normal), np.size(fraud_name_uid_black)

(20218730, 294745)

In [19]:
# 평균 비교
np.mean(fraud_name_uid_normal['changed']), np.mean(fraud_name_uid_black['changed'])

(1.4138885083286636, 1.9089721623776486)

In [20]:
# 등분산 검정
leveneTest = stats.levene(fraud_name_uid_normal['changed'], fraud_name_uid_black['changed'])

leveneTest

LeveneResult(statistic=9065.690648394393, pvalue=0.0)

In [21]:
# T-test
tTestResult = stats.ttest_ind(fraud_name_uid_normal['changed'], fraud_name_uid_black['changed'], equal_var=False)

tTestResult

Ttest_indResult(statistic=-63.14315728134901, pvalue=0.0)

### operation, 운영팀에서 처리한 유저

In [22]:
fraud_name_uid_normal = fraud_name_uid[fraud_name_uid['block_by_operation'] == 0]
fraud_name_uid_black = fraud_name_uid[fraud_name_uid['block_by_operation'] == 1]

In [23]:
# 샘플 수 비교
np.size(fraud_name_uid_normal), np.size(fraud_name_uid_black)

(20446575, 66900)

In [24]:
# 평균 비교
np.mean(fraud_name_uid_normal['changed']), np.mean(fraud_name_uid_black['changed'])

(1.4194663409397417, 1.8903587443946188)

In [25]:
# 등분산 검정
leveneTest = stats.levene(fraud_name_uid_normal['changed'], fraud_name_uid_black['changed'])

leveneTest

LeveneResult(statistic=1879.203176330235, pvalue=0.0)

In [26]:
# T-test
tTestResult = stats.ttest_ind(fraud_name_uid_normal['changed'], fraud_name_uid_black['changed'], equal_var=False)

tTestResult

Ttest_indResult(statistic=-29.09745119728514, pvalue=1.471915339519034e-180)

## black user와 white user의 휴대폰 변경 횟수 T-Test

In [27]:
read_path = '../csv/changed_phone_raw.csv'
fraud_phone_raw = pd.read_csv(read_path)

fraud_phone_raw.tail()

,date,uid
434520,2019-07-04 11:54:52,5000124
434521,2019-07-04 11:59:15,9245231
434522,2019-07-04 11:59:15,7375046
434523,2019-07-04 11:59:45,9325895
434524,2019-07-04 11:59:45,5000124


In [28]:
fraud_phone = fraud_phone_raw.groupby(['uid'], as_index=False).agg({'date': 'count'})

fraud_phone = fraud_phone.rename(columns={'date': 'changed'})
fraud_phone['uid'] = fraud_phone['uid'].astype(str)

fraud_phone.tail()

,uid,changed
351970,9325547,1
351971,9325556,1
351972,9325757,1
351973,9325864,1
351974,9325895,1


In [29]:
fraud_phone_uid = pd.merge(fraud_phone, fraud_uid, on=['uid'], how='left')

fraud_phone_uid = fraud_phone_uid.drop(['black_source'], axis=1).fillna(0)

fraud_phone_uid.tail()

,uid,changed,help,block,block_by_operation
352755,9325547,1,0.0,0.0,0.0
352756,9325556,1,0.0,0.0,0.0
352757,9325757,1,0.0,0.0,0.0
352758,9325864,1,0.0,0.0,0.0
352759,9325895,1,0.0,0.0,0.0


### help, 상담센터 신고된 유저

In [30]:
fraud_phone_uid_normal = fraud_phone_uid[fraud_phone_uid['help'] == 0]
fraud_phone_uid_black = fraud_phone_uid[fraud_phone_uid['help'] == 1]

In [31]:
# 샘플 수 비교
np.size(fraud_phone_uid_normal), np.size(fraud_phone_uid_black)

(1671050, 92750)

In [32]:
# 평균 비교
np.mean(fraud_phone_uid_normal['changed']), np.mean(fraud_phone_uid_black['changed'])

(1.219724125549804, 1.5118059299191375)

In [33]:
# 등분산 검정
leveneTest = stats.levene(fraud_phone_uid_normal['changed'], fraud_phone_uid_black['changed'])

leveneTest

LeveneResult(statistic=2796.9433308772304, pvalue=0.0)

In [34]:
# T-test
tTestResult = stats.ttest_ind(fraud_phone_uid_normal['changed'], fraud_phone_uid_black['changed'], equal_var=False)

tTestResult

Ttest_indResult(statistic=-32.8589930646798, pvalue=1.9010494159961004e-230)

### block, 영구 제재된 유저

In [35]:
fraud_phone_uid_normal = fraud_phone_uid[fraud_phone_uid['block'] == 0]
fraud_phone_uid_black = fraud_phone_uid[fraud_phone_uid['block'] == 1]

In [36]:
# 샘플 수 비교
np.size(fraud_phone_uid_normal), np.size(fraud_phone_uid_black)

(1718630, 45170)

In [37]:
# 평균 비교
np.mean(fraud_phone_uid_normal['changed']), np.mean(fraud_phone_uid_black['changed'])

(1.2290283539796234, 1.4654638034093426)

In [38]:
# 등분산 검정
leveneTest = stats.levene(fraud_phone_uid_normal['changed'], fraud_phone_uid_black['changed'])

leveneTest

LeveneResult(statistic=913.1073165499738, pvalue=2.5073199830925078e-200)

In [39]:
# T-test
tTestResult = stats.ttest_ind(fraud_phone_uid_normal['changed'], fraud_phone_uid_black['changed'], equal_var=False)

tTestResult

Ttest_indResult(statistic=-23.524199133055664, pvalue=6.451233280996861e-119)

### operation, 운영팀에서 처리한 유저

In [40]:
fraud_phone_uid_normal = fraud_phone_uid[fraud_phone_uid['block_by_operation'] == 0]
fraud_phone_uid_black = fraud_phone_uid[fraud_phone_uid['block_by_operation'] == 1]

In [41]:
# 샘플 수 비교
np.size(fraud_phone_uid_normal), np.size(fraud_phone_uid_black)

(1750060, 13740)

In [42]:
# 평균 비교
np.mean(fraud_phone_uid_normal['changed']), np.mean(fraud_phone_uid_black['changed'])

(1.2333234289110087, 1.4592430858806404)

In [43]:
# 등분산 검정
leveneTest = stats.levene(fraud_phone_uid_normal['changed'], fraud_phone_uid_black['changed'])

leveneTest

LeveneResult(statistic=257.75457128903776, pvalue=5.553551371173039e-58)

In [44]:
# T-test
tTestResult = stats.ttest_ind(fraud_phone_uid_normal['changed'], fraud_phone_uid_black['changed'], equal_var=False)

tTestResult

Ttest_indResult(statistic=-12.649150895428923, pvalue=1.073359514808459e-35)

## black user와 white user의 리뷰 작성 횟수 T-Test

In [45]:
read_path = '../csv/review_writer.csv'
fraud_review_writer_raw = pd.read_csv(read_path)

fraud_review_writer_raw.tail()

,time,writer_uid
5902957,2019-07-04 12:22:48,6439210
5902958,2019-07-04 12:23:38,4204619
5902959,2019-07-04 12:23:42,8994648
5902960,2019-07-04 12:23:47,1045672
5902961,2019-07-04 12:27:13,7810960


In [46]:
fraud_review_writer = fraud_review_writer_raw.groupby(['writer_uid'], as_index=False).agg({'time': 'count'})

fraud_review_writer = fraud_review_writer.rename(columns={'time': 'writed', 'writer_uid': 'uid'})
fraud_review_writer['uid'] = fraud_review_writer['uid'].astype(str)

fraud_review_writer.tail()

,uid,writed
1110787,9316109,1
1110788,9320668,1
1110789,9321110,1
1110790,9321727,1
1110791,9322225,1


In [47]:
fraud_review_writer_uid = pd.merge(fraud_review_writer, fraud_uid, on=['uid'], how='left')

fraud_review_writer_uid = fraud_review_writer_uid.drop(['black_source'], axis=1).fillna(0)

fraud_review_writer_uid.tail()

,uid,writed,help,block,block_by_operation
1112416,9316109,1,0.0,0.0,0.0
1112417,9320668,1,0.0,0.0,0.0
1112418,9321110,1,0.0,0.0,0.0
1112419,9321727,1,0.0,0.0,0.0
1112420,9322225,1,0.0,0.0,0.0


### help, 상담센터 신고된 유저

In [48]:
fraud_review_writer_uid_normal = fraud_review_writer_uid[fraud_review_writer_uid['help'] == 0]
fraud_review_writer_uid_black = fraud_review_writer_uid[fraud_review_writer_uid['help'] == 1]

In [49]:
# 샘플 수 비교
np.size(fraud_review_writer_uid_normal), np.size(fraud_review_writer_uid_black)

(5238555, 323550)

In [50]:
# 평균 비교
np.mean(fraud_review_writer_uid_normal['writed']), np.mean(fraud_review_writer_uid_black['writed'])

(4.822710652078674, 13.474748879616751)

In [51]:
# 등분산 검정
leveneTest = stats.levene(fraud_review_writer_uid_normal['writed'], fraud_review_writer_uid_black['writed'])

leveneTest

LeveneResult(statistic=15048.115826535262, pvalue=0.0)

In [52]:
# T-test
tTestResult = stats.ttest_ind(fraud_review_writer_uid_normal['writed'], fraud_review_writer_uid_black['writed'], equal_var=False)

tTestResult

Ttest_indResult(statistic=-54.20249604555359, pvalue=0.0)

### block, 영구 제재된 유저

In [53]:
fraud_review_writer_uid_normal = fraud_review_writer_uid[fraud_review_writer_uid['block'] == 0]
fraud_review_writer_uid_black = fraud_review_writer_uid[fraud_review_writer_uid['block'] == 1]

In [54]:
# 샘플 수 비교
np.size(fraud_review_writer_uid_normal), np.size(fraud_review_writer_uid_black)

(5396740, 165365)

In [55]:
# 평균 비교
np.mean(fraud_review_writer_uid_normal['writed']), np.mean(fraud_review_writer_uid_black['writed'])

(5.2216662281303154, 8.731079732712484)

In [56]:
# 등분산 검정
leveneTest = stats.levene(fraud_review_writer_uid_normal['writed'], fraud_review_writer_uid_black['writed'])

leveneTest

LeveneResult(statistic=1203.5252309507368, pvalue=1.4476874595009652e-263)

In [58]:
# T-test
tTestResult = stats.ttest_ind(fraud_review_writer_uid_normal['writed'], fraud_review_writer_uid_black['writed'], equal_var=False)

tTestResult

Ttest_indResult(statistic=-28.92924257275353, pvalue=8.06609605512334e-182)

### operation, 운영팀에서 처리한 유저

In [59]:
fraud_review_writer_uid_normal = fraud_review_writer_uid[fraud_review_writer_uid['block_by_operation'] == 0]
fraud_review_writer_uid_black = fraud_review_writer_uid[fraud_review_writer_uid['block_by_operation'] == 1]

In [60]:
# 샘플 수 비교
np.size(fraud_review_writer_uid_normal), np.size(fraud_review_writer_uid_black)

(5530980, 31125)

In [61]:
# 평균 비교
np.mean(fraud_review_writer_uid_normal['writed']), np.mean(fraud_review_writer_uid_black['writed'])

(5.298332302774553, 10.243212851405623)

In [62]:
# 등분산 검정
leveneTest = stats.levene(fraud_review_writer_uid_normal['writed'], fraud_review_writer_uid_black['writed'])

leveneTest

LeveneResult(statistic=502.5365131726051, pvalue=2.8236969570967546e-111)

In [63]:
# T-test
tTestResult = stats.ttest_ind(fraud_review_writer_uid_normal['writed'], fraud_review_writer_uid_black['writed'], equal_var=False)

tTestResult

Ttest_indResult(statistic=-15.137973808885832, pvalue=7.196922087061975e-51)